In [1]:
import pandas as pd
import re
import gc
import numpy as np
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.externals import joblib
import xgboost as xgb

In [10]:
df_bids = pd.read_csv("bids.csv")

In [11]:
df_bids = df_bids.replace({' ': ''}, regex = True) #remove spaces

In [44]:
df_bids_sorted = df_bids.sort_values(['bidder_id', 'time'], ascending = [True, True])

In [16]:
#dataframe for aggregated bid data
bids = pd.DataFrame(data = df_bids_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = df_bids_sorted['bidder_id'].unique())

In [17]:
timediff = df_bids_sorted.groupby('bidder_id')['time'].diff()
timediff_str = timediff.astype(str).fillna('')
df_bids_sorted['timediff_num'] = timediff
df_bids_sorted['timediff'] = timediff_str

In [39]:
#turn feature sequences into text
text_cols = ['auction', 'merchandise', 'device', 'timediff', 'country', 'ip', 'url']
for var in text_cols:
    df_bids_sorted[var] = var + "_" + df_bids_sorted[var].fillna("")
    text_str = var + '_text'
    count_str = var + '_nunique_num'
    bids[text_str] = df_bids_sorted.groupby('bidder_id')[var].apply(lambda x: "%s" % ' '.join(x))
    bids[count_str]  = df_bids_sorted.groupby('bidder_id')[var].nunique()

NameError: name 'df_bids_sorted' is not defined

In [19]:
# MIN()_MAX_TIME DIFFERENCES.....
max_time = df_bids_sorted.groupby('bidder_id')['time'].max()
bids['maxtime_num'] = max_time
min_time = df_bids_sorted.groupby('bidder_id')['time'].min()
bids['mintime_num'] = min_time
max_diff = df_bids_sorted.groupby('bidder_id')['timediff_num'].max()
max_diff = max_diff.fillna(max_diff.mean())
bids['maxdiff_num'] = max_diff
min_diff = df_bids_sorted.groupby('bidder_id')['timediff_num'].max()
min_diff = min_diff.fillna(min_diff.mean())
bids['mindiff_num'] = min_diff
range_diff = max_diff - min_diff
bids['rangediff_num'] = range_diff
mean_diff = df_bids_sorted.groupby('bidder_id')['timediff_num'].mean()
mean_diff = mean_diff.fillna(mean_diff.mean())
bids['meandiff_num'] = mean_diff
median_diff = df_bids_sorted.groupby('bidder_id')['timediff_num'].median()
median_diff = median_diff.fillna(median_diff.mean())
bids['mediandiff_num'] = median_diff
for q in [0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9]:
    q_string = 'diff_quantile_num_' + str(q).replace('.', '_')
    q_temp = df_bids_sorted.groupby('bidder_id')['timediff_num'].quantile(q)
    q_temp = q_temp.fillna(q_temp.mean())
    bids[q_string] = q_temp

In [20]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_combo = df_train.append(df_test)
df_combo['address_text'] = 'address_' + df_combo['address'].fillna('')
df_combo['account_text'] = 'account_' + df_combo['payment_account'].fillna('')
df_combo = df_combo.merge(bids, how = 'left', left_on = ['bidder_id'], right_on = ['bidder_id'])

In [21]:
del df_train
del df_test
del df_bids
del df_bids_sorted
del bids
gc.collect();

In [34]:
num_cols = filter(re.compile('num').search, df_combo.columns)
text_cols = filter(re.compile('text').search, df_combo.columns)
for col in num_cols:
    df_combo[col] = df_combo[col].fillna(df_combo[col].mean())
for col in text_cols:
    df_combo[col] = df_combo[col].fillna('')

In [35]:
sample = pd.read_csv('sampleSubmission.csv')
test_dat = df_combo[df_combo.bidder_id.isin(sample.bidder_id)]
#test
print (sample.bidder_id.values==test_dat['bidder_id'].values)

[ True  True  True ...,  True  True  True]


In [38]:
train_dat = df_combo[~pd.isnull(df_co)]
y = train_dat.columns.values
xtrain = train_dat[num_cols].values
xtest = test_dat[num_cols].values
col_names = num_cols


ValueError: Item wrong length 35 instead of 6713.